In [2]:
RRR = RealField(40);
P.<x> = PolynomialRing(ZZ);


#Generates a random polynomial of degree and norm bounded by N and no
def Rand_Pol_Fixed_Norm(N,no):
    v = vector(ZZ,N);
    for i in range(N):
        v[i] = randint(-1000*no,1000*no);
    norm = v*v;
    for i in range(N):
        v[i] = (v[i]*no)//isqrt(norm);
    f = sum(v[i]*x^i for i in range(N));
    return f;


#For fixed f,g,N,q, generates the NTRU polynomials F,G associated by f,g
def Derivation_FG(f,g,N,q):
    phi = x^N+1;
    (R_f, rho_f, iphi) = xgcd(f,phi);
    (R_g, rho_g, iphi) = xgcd(g,phi);
    assert(R_f.degree() == 0);
    assert(R_g.degree() == 0);
    (pgcd,alpha,beta) = xgcd(R_f[0],R_g[0]);
    if (pgcd != 1):
        print ("pgcd =", pgcd)
        print ("The GCD of R_f and R_g is different of 1.")
    assert(pgcd == 1);
    F = -q*beta*rho_g;
    G = q*alpha*rho_f;
    k = Compute_k(f,g,F,G,N);
    iter = 0;
    while (k!= 0):
        F = (F - k*f).quo_rem(phi)[1];
        G = (G - k*g).quo_rem(phi)[1];
        k = Compute_k(f,g,F,G,N);
    return(F,G);


#Tests if f,g can generate a NTRU lattice
def Test(f,g,N,q):
    phi = x^N+1;
    (R_f, rho_f, iphi) = xgcd(f,phi);
    (R_g, rho_g, iphi) = xgcd(g,phi);
    assert(R_f.degree() == 0);
    assert(R_g.degree() == 0);
    (pgcd,alpha,beta) = xgcd(R_f[0],R_g[0]);
    if (pgcd != 1):
        return False;
    return True;


#Returns f(1/x) mod (x^N+1)
def Reverse(f,N):
    g = sum( f[i]*(x^(N-i)) for i in [1..N-1] );
    return f[0] - g;


#Compute the polynomial k used to reduce F,G
#(See the end of Appendix A in "NTRUSign: Digital Signatures using the NTRU Lattice")
def Compute_k(f,g,F,G,N):
    RRR=RealField(350);
    phi = (x^N+1);
    FB = Reverse(F,N);
    GB = Reverse(G,N);
    fb = Reverse(f,N);
    gb = Reverse(g,N);
    num = (fb*F+gb*G).quo_rem(phi)[1];
    den = (f*fb+g*gb).quo_rem(phi)[1];
    (a,iden,iphi) = xgcd(den,x^N+1);
    k0 = (num*iden).quo_rem(phi)[1];
    k = sum( (k0[i]//a[0])*x^i for i in [0..N-1]  );
    k = k.change_ring(ZZ);
    return k;


def Rotate(v,k):
    w = list(v);
    for i in range(k):
        w.insert(0,-w.pop());
    return vector(w);


#Returns the Anticirculant matrix A_N(f) generated by, f, x^k.f, ..., x^((N-1)k).f
def AC(f,N,k):
    u = f.coefficients();
    while(len(u)<N):
        u.append(0);
    A = matrix(ZZ,N);
    z = vector(u);
    for i in range(N):
        A[i] = z;
        z = Rotate(z,k);
    return A;


#Tests if f is invertible mod X^N+1 mod q
def Is_Invertible(f,N,q):
    (pgcd,u,v) = xgcd(f,x^N+1);
    rep = gcd(pgcd,q);
    return (rep==1);


#Computes the inverse of f mod X^N+1 mod q
def Inverse(f,N,q):
    (pgcd,u,v) = xgcd(f,x^N+1);
    p_1 = inverse_mod(pgcd[0],q);
    u = p_1*u;
    u = u.quo_rem(q)[1];
    return u;


#Computes h = g/f mod X^N+1 mod q
def h_from_fg(f,g,N,q):
    phi = x^N+1;
    f_1 = Inverse(f,N,q);
    h = ((f_1*g).quo_rem(phi)[1]).quo_rem(q)[1];
    return h;


#Returns the NTRU secret basis generated by f,g
def NTRU_Secret_Basis(f,g,N,q):
    (F,G) = Derivation_FG(f,g,N,q);
    #print (f*G - g*F == q)
    print (f)
    print (g)
    print (F)
    print (G)
    A = AC(f,N,1);
    B = AC(g,N,1);
    C = AC(F,N,1);
    D = AC(G,N,1);
    E = block_matrix([[A,B],[C,D]]);
    return E;


#Returns the NTRU public basis generated by f,g
def NTRU_Public_Basis(f,g,N,q):
    phi = x^N+1;
    h = h_from_fg(f,g,N,q);
    A = identity_matrix(ZZ,N);
    B = AC(h,N,1);
    C = zero_matrix(ZZ,N);
    D = q*identity_matrix(ZZ,N);
    E = block_matrix([[A,B],[C,D]]);
    return E;


#Push-button procedure for generating the public and private bases for a NTRU lattice
#The expected norms of f,g is hardcoded ('norm') but you can change it
def Keygen(N,q):
    norm = isqrt(q)//2;
    Rep = False;
    while(Rep==False):
        f = Rand_Pol_Fixed_Norm(N,norm);
        g = Rand_Pol_Fixed_Norm(N,norm);
        Rep = Test(f,g,N,q);
        if(Rep==True):
            Rep = Is_Invertible(f,N,q);
    Sk = NTRU_Secret_Basis(f,g,N,q);
    Pk = NTRU_Public_Basis(f,g,N,q);
    return (Sk,Pk);

def gen_NTRU_fgFG(N, q):
    norm = isqrt(q)//2;
    Rep = False;
    while(Rep==False):
        f = Rand_Pol_Fixed_Norm(N,norm);
        g = Rand_Pol_Fixed_Norm(N,norm);
        Rep = Test(f,g,N,q);
        if(Rep==True):
            Rep = Is_Invertible(f,N,q);
    (F,G) = Derivation_FG(f,g,N,q);
    return f,g,F,G

def Inverse(f,N,q):
    (pgcd,u,v) = xgcd(f,x^N+1);
    p_1 = inverse_mod(pgcd[0],q);
    u = p_1*u;
    u = u.quo_rem(q)[1];
    return u;


#Computes h = g/f mod X^N+1 mod q
def h_from_fg(f,g,N,q):
    phi = x^N+1;
    f_1 = Inverse(f,N,q);
    h = ((f_1*g).quo_rem(phi)[1]).quo_rem(q)[1];
    return h;


In [3]:
def random_prime(k):
    while True:
        p = ZZ.random_element(2**k, 2**(k+1))
        if p.is_prime() == True:
            return p
 

class Shamir():
    
    def __init__(self, t, n):
        self._t = t
        self._n = n

        
    def GC(self, k):
        ## (a) pick a prime p
        p = random_prime(k)
        assert p > self._n
        
        ## (b) pick uniformly at random n distinct non-zero elements alpha from Zp*
        Zp = Zmod(p)
        alpha = []
        for i in range(self._n):
            alpha.append(Zp.random_element())
        
        return (p, alpha)
        # return x=(p, alpha)
        
        
    def DS(self, p, alpha, s):
        ## generate t-1 uniformly random elements a=(a_1,...,a_(t-1)) from Zp
        Zp = Zmod(p)
        a = [s]
        for i in range(self._t - 1):
            a.append(Zp.random_element()) 
            
        # build the polynomial a(x)=s+a_1*x+...+a_(t-1)*x^(t-1) from Zp[x;t-1]
        F = FiniteField(p)
        P = F['x']
        ax = P(a)
        
        # generate shares sigma_i
        shares = []
        for i in range(self._n):
            sigma_i = ax(alpha[i]) % p
            shares.append([alpha[i], sigma_i])
        return shares
        
        
    def SC(self, p, shares):
        assert len(shares) == self._t
        F = FiniteField(p)
        P = F['x']
        print (P.lagrange_polynomial(shares))
        
        
# s = 100
# k = 33

#shaTSS = Shamir(t, n)
#p, alpha = shaTSS.GC(k)
# shares = shaTSS.DS(p, alpha, s)
# shaTSS.SC(p, shares[:3] + [[123, 123]])
# shaTSS.SC(p, shares[:4])
    


In [15]:
NN = 251
q = 128
df = 73
dg = 71
t = 3
n = 5 # number of users
T = 'transpose'

R1.<x> = PolynomialRing(ZZ)
ideal = x**NN - 1
Rx = R1.quotient_by_principal_ideal(ideal)

print (Rx)

def gen_fg(R, d):
    print (d)
    while True:
        f = R.random_element()._polynomial
        coef = f.coefficients()
        l = len([x for x in coef if x == 1])
        if l == d:
            print (f)
            return f
    
def gen_f(R, df):
    f = R.random_element()._polynomial
    coefs = [abs(x) for x in f.coefficients()]
    coefs_count = len(coefs)
    
    for i in range(coefs_count):
        if coefs[i] == 0:
            coefs[i] = -1
        else:
            coefs[i] = 0
    
    for i in range(df):
        while True:
            a = randint(0, coefs_count - 1)
            if coefs[a] != 1:
                coefs[a] = 1
                break
                
    return R(coefs)._polynomial

# f = gen_f(Rt, df)
# g = gen_f(Rt, dg)
        
# #f = gen_fg(Rt, df) #Rt.random_element()._polynomial
# #g = gen_fg(Rt, dg) #Rt.random_element()._polynomial

# # while True:
# #     F = Rt.random_element()
# #     G = Rt.random_element()
# #     if f*G - F*g == Rt(q):
# #         break
# # print ('wow')

# f_res = f.resultant(t**NN-1)
# g_res = g.resultant(t**NN-1)

# ffff = 0
# for i in range(NN):
#     ffff += t**i
# ffff = Rt(ffff)._polynomial

# Rf = f_res % ffff
# Rg = g_res % ffff

# _,alpha,beta = xgcd(Rf, Rg)

# def gen_p(f):
#     test = 1
#     for i in range(2, NN):
#         test *= Rt(f(t**i))
#         test = test.mod(ffff)
#     return test._polynomial

# pf = gen_p(f)
# pg = gen_p(g)

# Zn = Zmod(q)
# R2.<t> = PolynomialRing(Zn)
# ideal2 = t**NN - 1
# Rt2 = R2.quotient_by_principal_ideal(ideal2)
    
# F = -q * beta * pg#)._polynomial
# G = q*alpha*pf#)._polynomial


# #Returns f(1/x) mod (x^N+1)
# def Reverse(f,N):
#     g = sum( f[i]*(t^(N-i)) for i in [1..N-1] );
#     return f[0] - g;


# #Compute the polynomial k used to reduce F,G
# #(See the end of Appendix A in "NTRUSign: Digital Signatures using the NTRU Lattice")
# def Compute_k(f,g,F,G,N):
#     RRR=RealField(350);
#     phi = (t^N+1);
#     FB = Reverse(F,N);
#     GB = Reverse(G,N);
#     fb = Reverse(f,N);
#     gb = Reverse(g,N);
#     num = (fb*F+gb*G).quo_rem(phi)[1];
#     den = (f*fb+g*gb).quo_rem(phi)[1];
#     (a,iden,iphi) = xgcd(den,t^N+1);
#     k0 = (num*iden).quo_rem(phi)[1];
#     k = sum( (k0[i]//a[0])*t^i for i in [0..N-1]  );
#     k = k.change_ring(ZZ);
#     return k;

# k = Compute_k(f,g,F,G,NN);
# iter = 0;
# while (k!= 0):
#     #print ((F - k*f))
#     F = (F - k*f).quo_rem(phi)[1];
#     G = (G - k*g).quo_rem(phi)[1];
#     k = Compute_k(f,g,F,G,N);
    
# print (F)
# QQ = f*G - g*F
# #print (QQ)
# #print (QQ % q)
# s = 0
# for el in QQ.coefficients():
#     s += el % q

class Member:
    shares = []
    def __init__(self, f,g,F,G):
        self.f = f
        self.g = g
        self.F = F
        self.G = G
    
    def calculate_params(self, N, q, T = 'transpose'):
        if T == 'standard':
            self.fs = F
        elif T == 'transpose':
            self.fs = g
        self.h = h_from_fg(f,g,N,q)
        
    def eval_k(self):
        # save unsigned values, remember it
        f_bin  = ''.join([bin(abs(x))[2:] for x in  self.f.list()])
        fs_bin = ''.join([bin(abs(x))[2:] for x in self.fs.list()])
        h_bin = ''.join([bin(abs(x))[2:] for x in self.h.list()])
        self.k = int(f_bin + fs_bin + h_bin, 2)
        
    def update_ms(self, m, s):
        self.m = m
        self.s = s
        
    
members = []
for i in range(n+1):
    f,g,F,G = gen_NTRU_fgFG(NN, q)
    m = Member(f,g,F,G)
    m.calculate_params(NN, q, T)
    m.eval_k()
    members.append(m)
    #members[i].shares = []
    
    

# shaTSS = Shamir(t, n)
# p, alpha = shaTSS.GC(7)
# for i in range(0, n+1):
#     shares = shaTSS.DS(p, alpha, members[i].k)
#     print (i, shares)
#     for i in range(0, n+1):
#         members[i].shares.append(shares[i-1])


shaTSS = Shamir(t, n)
p, alpha = shaTSS.GC(2048)
#print (p)
print (members[0].k)
shares = shaTSS.DS(p, alpha, members[0].k)
#print (shares)
for i in range(1, n+1):
    members[i].shares = shares[i-1]



Univariate Quotient Polynomial Ring in xbar over Integer Ring with modulus x^251 - 1
321480555898198226752917908452528014873937024355415404470776671061518256296432536776604134586481095792188850824863389760201466781283692719539703702351681247860977823414719809320899202894914058969001687256378527687912535765214278260409607566451323209262972520207271368102758619443882942224744765420497043908389543626407396428576789528491683974296837785627674539901028735676834456175841068355400497751001893304386632192520456607093010735826547382139599890465276013221210118228001322539002652337097468557169642798122818308879812620779661969576740124214008032273168830876212334043727165256057193332


In [17]:
# (B) Partial Signature Generator

from hashlib import md5

D = 'testmsg123_'

def normilize_coeffs(f):
    R = f.parent()
    normilized_coeffs = []
    for i in range(f.degree()):
        normilized_coeffs.append(int(round(x[i].n())))
    return R(normilized_coeffs)

members[0].m = int(md5((D+'0').encode('utf-8')).hexdigest(),16)
members[0].s = 0

#random_members = [1, 3, 4]

for i in range(1, n+1):
    h, f, fs = members[i].h, members[i].f, members[i].fs
    h_prev, m_prev, s_prev = members[i-1].h, members[i-1].m, members[i-1].s
    
    x = normilize_coeffs(-(1/q)*m_prev*fs)
    y = normilize_coeffs(-(1/q)*m_prev*f)
    s = x*f + y*fs
    m = (s * (h - h_prev)).quo_rem(q)[1]
    s = s + s_prev
    
    members[i].m = m
    members[i].s = s

    

In [18]:
# (C) Threshold Signature Generator
h, m, s = members[n].h, members[n].m, members[n].s
p
#print (p)
shares = [m.shares for m in members[1:]]
#print (shares)
shaTSS.SC(p, shares[1:4])

995257060848212365601632365675191138384538896608587345345384911388418452663210811478146349772718578423531901950497367680975422345359374785332952171517200002011241286567657834368994999902850055809772457629184038525441401470550610557912145885054535763540431723046604313823197178742612773598406984577020495462417069146033589860272787539624813342145501056032475252507787531992407668493907411565284143933614095808386003769108214742199114379160953107428581468217332022127711451614592963780219170662415555927712709347967098303016946419140917072283178930508060576715240993732039196881673963859279120748578795856553170279254*x^2 + 183125899322092900308256271616379934949354050267195021772477053813907668799547679085070046141898793643354865480612602444054581454800403501158133836848078295992170697473832725342312504974663407935854841044943370863096348472858161158775511077031461588521053741513119276624808578894724147837005085185635269512604644214518845867044248562077607932396017135829577900621392717008407179

In [23]:
k = 321480555898198226752917908452528014873937024355415404470776671061518256296432536776604134586481095792188850824863389760201466781283692719539703702351681247860977823414719809320899202894914058969001687256378527687912535765214278260409607566451323209262972520207271368102758619443882942224744765420497043908389543626407396428576789528491683974296837785627674539901028735676834456175841068355400497751001893304386632192520456607093010735826547382139599890465276013221210118228001322539002652337097468557169642798122818308879812620779661969576740124214008032273168830876212334043727165256057193332
h_bin = ''.join([bin(abs(x))[2:] for x in members[0].h.list()])
f_bin = ''.join([bin(abs(x))[2:] for x in members[0].f.list()])
g_bin = ''.join([bin(abs(x))[2:] for x in members[0].g.list()])
truncation = len(f_bin) + len(g_bin)
recovered_k = bin(k)
print (recovered_k[truncation-1:])


0b11000000011010101011101001010110011001110011110100111000101000100010010100110100011000011010111111010000101110100010000001001011101011010011100111111010010110011000010010111111111111011110101011000011001100111101010100100000110001110110011100100010100011000111101101001001110001101011001100001010101110111101001110011110110001100010001111000111011111001010000101100110100110001110111000010001011010101001100110111010110010101001000000010000001101000010000111110011111110110100001001111101100101101110001111000010101001110010101010011011110110100001011100111011110100111010011010111011011001011101101101110111111111110110000101001001111000100111101000110100011101011111010011010101001010101001111001110110000110110101010011100101111111010101001110110100011011011101010000001101000110101111000110010011011101101011111101101010001110011000001101101110110111101111101101000111001011011101110101101101001011111101101010110001100101111011110111101111111001011001010110001101000010111011011101100101010100

In [26]:
h, m, s = members[n].h, members[n].m, members[n].s
f = members[0].f
fs = members[0].fs

x = normilize_coeffs(-(1/q)*m*fs)
y = normilize_coeffs(-(1/q)*m*f)
s0 = x*f + y*fs
s_res = s + s0
print (s_res)


-x^251 - x^250 - x^249 - 2*x^247 - x^246 - x^244 - x^243 - x^242 - x^241 - x^240 - x^239 - x^238 - x^236 - x^235 - x^234 - x^233 - 2*x^232 - x^230 - x^228 - 2*x^226 - 2*x^225 - x^223 - x^222 - x^221 - x^219 - x^217 - x^215 - x^214 - 2*x^213 - x^212 - x^210 - x^209 - 2*x^208 - x^206 - x^205 - 2*x^204 - x^203 - 2*x^202 - x^200 - 2*x^199 - x^198 - 2*x^196 - 2*x^195 - x^194 - x^193 - x^191 - x^190 - x^187 - 2*x^185 - x^184 - x^183 - x^182 - x^181 - 2*x^180 - x^179 - x^178 - x^176 - x^175 - 2*x^174 - x^171 - x^170 - x^169 - x^168 - x^166 - x^165 - x^163 - 2*x^162 - x^160 - 2*x^159 - x^157 - x^156 - 2*x^154 - x^153 - 2*x^152 - x^151 - x^150 - x^148 - 2*x^147 - 2*x^145 - x^144 - x^143 - 2*x^142 - x^141 - 2*x^140 - x^138 - 2*x^137 - 2*x^136 - x^135 - x^134 - 2*x^133 - x^131 - x^130 - x^128 - x^126 - x^125 - x^124 - x^122 - 2*x^121 - x^120 - 2*x^119 - 2*x^118 - x^117 - x^115 - x^113 - 2*x^109 - 2*x^108 - x^106 - x^104 - x^103 - x^102 - 2*x^101 - x^100 - x^98 - x^97 - x^95 - 2*x^94 - x^93 - x^92